### Prova con due groupby 

In [1]:
import pandas as pd
import numpy as np
import timeit as t
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy import optimize

In [2]:
dataset=pd.read_csv("data_000000.txt")
dataset['TIME']=dataset['BX_COUNTER']*25 + dataset['TDC_MEAS']*25/30 
fpga=dataset['FPGA']==1 
channel=(dataset['TDC_CHANNEL']>64)&(dataset['TDC_CHANNEL']<=128) 
trigger=dataset['TDC_CHANNEL'] == 139 

detectors=[~fpga&~channel,~fpga&channel,fpga&~channel,fpga&channel] 
for i in range(len(detectors)) : 
    dataset.loc[detectors[i],'DETECTOR']=i+1 
dataset.loc[trigger,'DETECTOR']=0 
rest = dataset['TDC_CHANNEL']%4 
dic_replace = {0:1, 2:2, 3:3, 1:4}
dataset['LAYER']= rest
dataset['LAYER'].replace(dic_replace, inplace=True)
dataset.loc[trigger,'LAYER']=0 
dataset = dataset.drop(['HEAD','FPGA'],axis=1) 
dataset

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER
0,116,1897414884,1533.0,24.0,38345.000000,4.0,1
1,71,1897414887,1650.0,21.0,41267.500000,4.0,3
2,67,1897414914,980.0,8.0,24506.666667,4.0,3
3,70,1897414922,1287.0,8.0,32181.666667,4.0,2
4,57,1897414922,2162.0,22.0,54068.333333,1.0,4
...,...,...,...,...,...,...,...
1292121,139,1920940255,2514.0,0.0,62850.000000,0.0,0
1292122,35,1920940255,2512.0,16.0,62813.333333,3.0,3
1292123,37,1920940255,2513.0,5.0,62829.166667,3.0,4
1292124,105,1920940255,2510.0,17.0,62764.166667,4.0,4


In [3]:
dataset['TDC_CHANNEL'].any()==137 and dataset['TDC_CHANNEL'].any()==138
# sono stati eliminati tutti i trigger associati agli scintillatori

False

In [4]:
dataset.groupby(['ORBIT_CNT']).groups

{192: [1292125], 1897414884: [0], 1897414887: [1], 1897414914: [2], 1897414922: [3, 4], 1897414934: [5, 6, 7, 8, 9, 10, 11], 1897414940: [12], 1897414952: [13], 1897414956: [14], 1897414964: [15], 1897414976: [16, 17], 1897414996: [18, 19], 1897415018: [20], 1897415055: [21], 1897415058: [22, 23], 1897415068: [24, 25, 26], 1897415077: [27], 1897415080: [28], 1897415082: [29], 1897415091: [30], 1897415099: [31, 32], 1897415110: [33, 34], 1897415118: [35], 1897415128: [36, 37], 1897415130: [38], 1897415131: [39], 1897415134: [40], 1897415137: [41], 1897415144: [42], 1897415145: [43], 1897415150: [44], 1897415154: [45], 1897415157: [46], 1897415160: [47], 1897415171: [48], 1897415176: [49], 1897415182: [50], 1897415194: [51], 1897415199: [52], 1897415202: [53], 1897415204: [54], 1897415207: [55, 56], 1897415227: [57], 1897415240: [58], 1897415247: [59], 1897415251: [60], 1897415254: [61, 62], 1897415269: [63], 1897415290: [64], 1897415293: [65], 1897415294: [66], 1897415295: [67], 1897415

In [27]:
#dataset.groupby(['ORBIT_CNT']).get_group('1897414934') #è lentissimo
prova = dataset.loc[dataset.groupby(['ORBIT_CNT']).groups.get(1920940255)] # questo ha detector diversi, vediamo di selezionare soltanto quelli che compaiono più volte
prova = prova.drop(prova[prova['DETECTOR']==0].index)  
indici = prova[prova['DETECTOR']!=(prova.DETECTOR.mode().values)[0]].index
prova = prova.drop(indici)
prova

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER
1292117,102,1920940255,2510.0,22.0,62768.333333,4.0,2
1292119,106,1920940255,2507.0,24.0,62695.000000,4.0,2
1292120,101,1920940255,2510.0,14.0,62761.666667,4.0,4
1292124,105,1920940255,2510.0,17.0,62764.166667,4.0,4


In [30]:
# DECIDO DI FARE UN GROUPBY PER ORBITE VISTA LA RARITA' DEL PASSAGGIO DI PARTICELLE 
# elimino gruppi di orbite con meno di tre hits 
# un'altra cosa prima
# def clean(group):
#     group1=group.drop(group[group['DETECTOR']==0].index)     
#     detector=group1.DETECTOR.mode().values 
#     indices=group1[group1['DETECTOR']!=detector[0]].index 
#     group=group.drop(indices) 
#     return group
# filtered1=dataset.groupby(['ORBIT_CNT'],group_keys=False)\
#                     .apply(clean)\
#                     .dropna()\
#                     .reset_index()\
#                     .drop(['index'],axis=1)
def cut(group):
    if group.shape[0]>2: return group
    else: return None                         
filtered1=dataset.groupby(['ORBIT_CNT'],group_keys=False)\
                    .apply(cut)\
                    .dropna()\
                    .reset_index()\
                    .drop(['index'],axis=1)
# elimino i gruppi di orbite prima senza trigger activation e poi con più di una trigger activation, motivo per il quale non mi aspetto più detector dentro uno stesso evento               
def trigger(group):
    if (group['DETECTOR']!=0).all(): return None
    else: return group                                    
filtered2=filtered1.groupby(['ORBIT_CNT'],group_keys=False)\
                    .apply(trigger)\
                    .dropna()\
                    .reset_index()\
                    .drop(['index'],axis=1)

def multiple_orbits(group):
    if group[group['DETECTOR']==0].shape[0]>1: return None
    else: return group                                     
filtered3=filtered2.groupby(['ORBIT_CNT'],group_keys=False)\
                    .apply(multiple_orbits)\
                    .dropna()\
                    .reset_index()\
                    .drop(['index'],axis=1)
# analizzo gruppo per gruppo quanti detector compaiono e mantengo soltanto quelle hits associate al detector che compare il maggior numero di volte. Questo cut andrebbe fatto prima di quello in cui selezioniamo i gruppi con più di tre hits perchè se eliminiamo da un gruppo delle hits corrispondenti ad un detector in minoranza allora potrebbe capitare che il gruppo rimanente contenga meno di tre hits, perchè erano più di tre ma comprese quelle del detector eliminato!
def clean(group):
   #group1=group.drop(group[group['DETECTOR']==0].index)     
   detector=group.DETECTOR.mode().values 
   indices=group[group1['DETECTOR']!=detector[0]].index 
   group=group.drop(indices) 
   return group
filtered4=filtered3.groupby(['ORBIT_CNT'],group_keys=False)\
                   .apply(clean)\
                   .dropna()\
                   .reset_index()\
                   .drop(['index'],axis=1)

In [31]:
filtered4.shape

(503050, 7)

In [32]:
filtered5 = filtered4.groupby(['ORBIT_CNT'],group_keys=False)\
                    .apply(cut)\
                    .dropna()\
                    .reset_index()\
                    .drop(['index'],axis=1)

In [33]:
filtered5.shape

(503044, 7)

In [36]:
def clean1(group):
   #group1=group.drop(group[group['DETECTOR']==0].index)     
   detector=group.DETECTOR.mode().values 
   indices=group[group['DETECTOR']!=detector[0]].index 
   group=group.drop(indices) 
   return group
ua = dataset.groupby(['ORBIT_CNT'],group_keys=False)\
                    .apply(clean1)\
                    .dropna()\
                    .reset_index()\
                    .drop(['index'],axis=1)
# IL PROBLEMA DI APPLICARE PRIMA CLEAN STA PROPRIO NELLA FUNZIONE PERCHE' CI SONO AD ESEMPIO EVENTI CHE CONTENGONO SOLTANTO L'ATTIVAZIONE DI TRIGGER E QUINDI APPENA LA ELIMINO NASCE UN PROBLEMA ALLORA: CAMBIO FUNZIONE CLEAN IN MODO CHE IN OGNI GRUPPO CI SIA SEMPRE ALMENO UNA HIT E QUINDI UN DETECTOR DA SCEGLIERE COME PREFERITO. (HO SOLTANTO ELIMINATO LA RIGA CON IL DROP DEI TRIGGER NELLA FUNZIONE CLEAN DI PARTENZA)


In [38]:
ua1 = ua.groupby(['ORBIT_CNT'],group_keys=False)\
                    .apply(cut)\
                    .dropna()\
                    .reset_index()\
                    .drop(['index'],axis=1)

In [39]:
ua1.shape

(533413, 7)

In [41]:
ua1.loc[ua1.groupby(['ORBIT_CNT']).groups.get(1920940255)]

,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,TIME,DETECTOR,LAYER
533409,102.0,1.920940e+09,2510.0,22.0,62768.333333,4.0,2.0
533410,106.0,1.920940e+09,2507.0,24.0,62695.000000,4.0,2.0
533411,101.0,1.920940e+09,2510.0,14.0,62761.666667,4.0,4.0
533412,105.0,1.920940e+09,2510.0,17.0,62764.166667,4.0,4.0
